EN ESTE NOTEBOOK, COMENZAMOS SABIENDO QUE SILICON VALLEY SE ENCUENTRA EN LA BAHÍA DE SAN FRANCISCO. POR ELLO ME VOY A CENTRAR EN EL ESTADO DE CALIFORNIA Y MAS EXACTAMENTE EN LAS CIUDADES DE SAN FRANCISCO Y SAN JOSE.

ADEMÁS NOS CENTRAREMOS EN AQUELLAS EMPRESAS TECNOLÓGICAS EXITOSAS QUE HAYAN RECAUDADO AL MENOS 1 MILLÓN DE DÓLARES.

In [530]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import re
import json, requests
import pandas as pd
import folium
import geopandas as gpd

In [356]:
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [416]:
db = client.get_database("IRONHACK")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'IRONHACK')

------------

Anteriormente en mongodb, habremos hecho una limpieza, en IRONHACK(donde se encuentran todas las compañías), y nos abremos quedado con las oficinas que se encuentran en San Francisco y en San José.


---------------

De las 1083 oficinas, 903 pertenecen a San Francisco. Por lo que nos guiaremos en buscar nuestra localización en San Francisco. En mongodb hemos realizado los filtros correspondientes donde lo reuniremos en un nuevo "collection" llamado "san_francisco".

In [541]:
res = db.san_francisco.find( {}, {"offices":1})
list(res.limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'offices': [{'description': None,
    'address1': '135 Mississippi St',
    'address2': '',
    'zip_code': '94107',
    'city': 'San Francisco',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.764726,
    'longitude': -122.394523}]}]

In [542]:
len(san_francisco)

903

In [543]:
category = list(db.san_francisco.find({"category_code":{"$ne":None}}))

In [544]:
set([i["category_code"] for i in category])

{'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'hardware',
 'health',
 'legal',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'search',
 'security',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web'}

-----------------

In [552]:
df = pd.DataFrame(db.san_francisco.find({"funding_rounds.raised_amount": {"$gt": 1000000}}, {"name": 1, "total_money_raised": 1, "offices.latitude": 1,"offices.longitude": 1, "_id": 0}))
df.head()

,name,total_money_raised,offices
0,Digg,$45M,"[{'latitude': 37.764726, 'longitude': -122.394..."
1,Twitter,$1.16B,"[{'latitude': 37.7768052, 'longitude': -122.41..."
2,StumbleUpon,$18.5M,"[{'latitude': 37.775196, 'longitude': -122.419..."
3,Scribd,$25.8M,"[{'latitude': 37.789634, 'longitude': -122.404..."
4,Powerset,$22.5M,"[{'latitude': 37.778613, 'longitude': -122.395..."


In [546]:
df.offices

0      [{'latitude': 37.764726, 'longitude': -122.394...
1      [{'latitude': 37.7768052, 'longitude': -122.41...
2      [{'latitude': 37.775196, 'longitude': -122.419...
3      [{'latitude': 37.789634, 'longitude': -122.404...
4      [{'latitude': 37.778613, 'longitude': -122.395...
                             ...                        
401    [{'latitude': 40.7523796, 'longitude': -74.005...
402    [{'latitude': 37.775196, 'longitude': -122.419...
403              [{'latitude': None, 'longitude': None}]
404    [{'latitude': 37.7869047, 'longitude': -122.40...
405    [{'latitude': 37.782475, 'longitude': -122.407...
Name: offices, Length: 406, dtype: object

In [553]:
lat = pd.DataFrame(db.san_francisco.find({"funding_rounds.raised_amount": {"$gt": 1000000}}, {"name": 1, "total_money_raised": 1, "offices.latitude": 1, "_id": 0}))
lat.head()

,name,total_money_raised,offices
0,Digg,$45M,[{'latitude': 37.764726}]
1,Twitter,$1.16B,[{'latitude': 37.7768052}]
2,StumbleUpon,$18.5M,"[{'latitude': 37.775196}, {'latitude': None}]"
3,Scribd,$25.8M,[{'latitude': 37.789634}]
4,Powerset,$22.5M,[{'latitude': 37.778613}]


In [554]:
"""función extract. 
url = https://stackoverflow.com/questions/41785095/apply-with-a-condition-on-a-pandas-dataframe-elementwise"""

def extract(df):
    
    return df.applymap(lambda x: x[0] if isinstance(x, list) else x)

Me va a llevar problemas la columna de offices por lo que voy a separarlas entre latitud y longitud.

In [555]:
lat = extract(lat)
lat.head()

,name,total_money_raised,offices
0,Digg,$45M,{'latitude': 37.764726}
1,Twitter,$1.16B,{'latitude': 37.7768052}
2,StumbleUpon,$18.5M,{'latitude': 37.775196}
3,Scribd,$25.8M,{'latitude': 37.789634}
4,Powerset,$22.5M,{'latitude': 37.778613}


In [556]:
latitude = lat.join(pd.DataFrame(lat["offices"].to_dict()).T)
latitude.head()

,name,total_money_raised,offices,latitude
0,Digg,$45M,{'latitude': 37.764726},37.764726
1,Twitter,$1.16B,{'latitude': 37.7768052},37.776805
2,StumbleUpon,$18.5M,{'latitude': 37.775196},37.775196
3,Scribd,$25.8M,{'latitude': 37.789634},37.789634
4,Powerset,$22.5M,{'latitude': 37.778613},37.778613


In [557]:
latitude.drop(["offices"], axis=1, inplace=True)
latitude.head()

,name,total_money_raised,latitude
0,Digg,$45M,37.764726
1,Twitter,$1.16B,37.776805
2,StumbleUpon,$18.5M,37.775196
3,Scribd,$25.8M,37.789634
4,Powerset,$22.5M,37.778613


------------

Longitud

In [558]:
lon = pd.DataFrame(db.san_francisco.find({"funding_rounds.raised_amount": {"$gt": 1000000}}, {"name": 1, "total_money_raised": 1, "offices.longitude": 1, "_id": 0}))
lon = extract(lon) #Le aplico la función de extracción de lista.
lon.head()

,name,total_money_raised,offices
0,Digg,$45M,{'longitude': -122.394523}
1,Twitter,$1.16B,{'longitude': -122.4169244}
2,StumbleUpon,$18.5M,{'longitude': -122.419204}
3,Scribd,$25.8M,{'longitude': -122.404052}
4,Powerset,$22.5M,{'longitude': -122.395289}


In [559]:
longitude = lon.join(pd.DataFrame(lon["offices"].to_dict()).T)
longitude.head()

,name,total_money_raised,offices,longitude
0,Digg,$45M,{'longitude': -122.394523},-122.394523
1,Twitter,$1.16B,{'longitude': -122.4169244},-122.416924
2,StumbleUpon,$18.5M,{'longitude': -122.419204},-122.419204
3,Scribd,$25.8M,{'longitude': -122.404052},-122.404052
4,Powerset,$22.5M,{'longitude': -122.395289},-122.395289


In [560]:
longitude = lon.join(pd.DataFrame(lon["offices"].to_dict()).T)
longitude.drop(["offices"], axis=1, inplace=True)
longitude.head()

,name,total_money_raised,longitude
0,Digg,$45M,-122.394523
1,Twitter,$1.16B,-122.416924
2,StumbleUpon,$18.5M,-122.419204
3,Scribd,$25.8M,-122.404052
4,Powerset,$22.5M,-122.395289


------------------

Unión de longitud a latitud.

In [561]:
companies = latitude.join(longitude["longitude"])
companies.head()

,name,total_money_raised,latitude,longitude
0,Digg,$45M,37.764726,-122.394523
1,Twitter,$1.16B,37.776805,-122.416924
2,StumbleUpon,$18.5M,37.775196,-122.419204
3,Scribd,$25.8M,37.789634,-122.404052
4,Powerset,$22.5M,37.778613,-122.395289


In [562]:
len(companies)

406

No me valen empresas que tengan algún valor NoNe.

In [563]:
companies = companies.dropna()
companies.head()

,name,total_money_raised,latitude,longitude
0,Digg,$45M,37.764726,-122.394523
1,Twitter,$1.16B,37.776805,-122.416924
2,StumbleUpon,$18.5M,37.775196,-122.419204
3,Scribd,$25.8M,37.789634,-122.404052
4,Powerset,$22.5M,37.778613,-122.395289


In [564]:
companies = gpd.GeoDataFrame(companies, geometry = gpd.points_from_xy(companies.longitude,companies.latitude))
companies.head()

,name,total_money_raised,latitude,longitude,geometry
0,Digg,$45M,37.764726,-122.394523,POINT (-122.39452 37.76473)
1,Twitter,$1.16B,37.776805,-122.416924,POINT (-122.41692 37.77681)
2,StumbleUpon,$18.5M,37.775196,-122.419204,POINT (-122.41920 37.77520)
3,Scribd,$25.8M,37.789634,-122.404052,POINT (-122.40405 37.78963)
4,Powerset,$22.5M,37.778613,-122.395289,POINT (-122.39529 37.77861)


In [568]:
companies.to_csv("companies.csv")

In [584]:
locations = companies[["latitude", "longitude"]]
locationlist = locations.values.tolist()
len(locationlist)

324

In [579]:
from folium.plugins import HeatMap

In [583]:
map2= folium.Map(location=[37.7716458,-122.43045],zoom_start = 13) 
heatmap = [[row["latitude"],row["longitude"]] for index, row in companies.iterrows()]
HeatMap(heatmap).add_to(map2)
map2

------------

In [582]:
map = folium.Map(location=[37.7736265,-122.4200582], zoom_start=12)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df["name"][point]).add_to(map)
map